## Dataset

En las siguientes celdas se importa el dataset y se prepara para la implementación de los modelos de aprendizaje.

*Split 70-30* -> 280-120

In [1]:
import pandas as pd
import kagglehub

path = kagglehub.dataset_download("dragonheir/logistic-regression")

print("Path to dataset files:", path)
dataset = pd.read_csv(r'C:\Users\FLopezP\.cache\kagglehub\datasets\dragonheir\logistic-regression\versions\1\Social_Network_Ads.csv')

Path to dataset files: C:\Users\FLopezP\.cache\kagglehub\datasets\dragonheir\logistic-regression\versions\1


In [2]:
# Female = 1
# Male = 0
aux = []
for i in range(len(dataset["Gender"])):
    if dataset["Gender"][i] == "Female":
        aux.append(1)
    else:
        aux.append(0)

dataset["GenderNum"] = aux
dataset = dataset.loc[:,["User ID", "Gender", "GenderNum", "Age", "EstimatedSalary", "Purchased"]]
dataset

,User ID,Gender,GenderNum,Age,EstimatedSalary,Purchased
0,15624510,Male,0,19,19000,0
1,15810944,Male,0,35,20000,0
2,15668575,Female,1,26,43000,0
3,15603246,Female,1,27,57000,0
4,15804002,Male,0,19,76000,0
...,...,...,...,...,...,...
395,15691863,Female,1,46,41000,1
396,15706071,Male,0,51,23000,1
397,15654296,Female,1,50,20000,1
398,15755018,Male,0,36,33000,0


In [57]:
# train test split
ds_train = dataset[0:280]
ds_test = dataset[280:400].reset_index()
ds_test = ds_test.drop(columns = ["index"])
labels_train = ds_train["Purchased"]
labels_test = ds_test["Purchased"].reset_index()

## Regresión Logística

Buscamos predecir si dadas 3 características (Género, Edad, Salario Estimado) se compra un elemento.

In [70]:
# Dimensiones:
# Vector de entrada: (M/F, Edad, Salario)
# Vector de pesos: (a, b, c)^T
# Sesgo: d
# z = Entrada.Pesos + Sesgo
# Proba = 1/(1+e^-z)

import numpy as np

# NOTAS: CREO QUE FALTA ESTANDARIZAR

def mix_char(ds):
    """
    ds = DataFrame, en este caso asumimos que tiene las características adecuadas del problema

    return: lista de características normalizadas
    """
    df = ds.drop(columns = ["User ID", "Gender"])
    for column in df.columns:
        if column != "GenderNum":
            df[column] = df[column] / df[column].max()
    
    char = []
    for i in range(len(ds)):
        aux = [df["GenderNum"][i], df["Age"][i], df["EstimatedSalary"][i]]
        arr_aux = np.array(aux)
        arr_aux = np.reshape(arr_aux, (1,3))
        char.append(arr_aux)
    return char

In [72]:
#Tenemos los conjuntos de entrenamiento ya agrupados. Ahora tenemos que realizar la multiplicación por los pesos.
train_char = mix_char(ds_train)
test_char = mix_char(ds_test)

print(train_char[0], test_char[0])

[[0.         0.31666667 0.12666667]] [[1.         0.98333333 0.61111111]]


In [44]:
pesos = np.random.rand(3,1)
bias = np.random.rand(1,1)
print(pesos, bias)

[[0.92849732]
 [0.97703612]
 [0.52910922]] [[0.80411763]]


In [73]:
z = np.dot(train_char[0], pesos) + bias
print(z)

[[1.1805329]]


**Ahora hay que automatizar la comparación del vector de pesos con todos los valores de entrenamiento**

**Posteriormente hay que ver cómo pingas vamos a hacer el descenso del gradiente**

## Red Neuronal

## K-Means